In [1]:
import vk_api
from vk_api.longpoll import VkLongPoll, VkEventType
from random import randrange
import json
import requests
import sqlite3
conn = sqlite3.connect("cinemas.db")
cursor = conn.cursor()

In [2]:
def write_msg(user_id, message, keyboard=None):
    random_id=randrange(-pow(2,63),pow(2,63)-1,1)
    vk.method('messages.send', {'user_id': user_id, 'message': message, 'random_id':random_id, 'keyboard':keyboard})

In [3]:
def halls(brand):
    brand_id = cursor.execute(f"select id from brand where name='{brand}'").fetchall()[0][0]
    li = list()
    for v in cursor.execute(f"select name from cinema_halls where brand_id='{brand_id}'").fetchall():
        li.append(v)
    return li

def dates(concret_hall):
    hall_id=cursor.execute(f"select id from cinema_halls where name='{concret_hall[0]}'").fetchall()[0][0]
    dates_list=list(set([v[0] for v in cursor.execute(f"select date from sessions where hall_id='{hall_id}'").fetchall()]))    
    return sorted(dates_list)

def conc_sessions(date, concret_hall):
    hall_id=cursor.execute(f"select id from cinema_halls where name='{concret_hall[0]}'").fetchall()[0][0]
    sessions=cursor.execute(f"select cinema_id from sessions where (hall_id='{hall_id}'and date='{date}') ").fetchall()
    for i, session in enumerate(sessions):
        sessions[i] = cursor.execute(f"select name from cinemas where id='{session[0]}'").fetchall()[0][0]
    return sorted(list(set(sessions)))

def data(brand, hall, date, film):
    hall_data = cursor.execute(f"select address, metro, phone, id from cinema_halls where name='{hall[0]}'").fetchall()[0]
    film_data = cursor.execute(f"select duration, genres, id from cinemas where name='{film}'").fetchall()[0]
    sessions_data = cursor.execute(f"select form, time from sessions where (cinema_id='{film_data[2]}' and hall_id='{hall_data[3]}' and date='{date}')").fetchall()
    info =f'''Сеть кинотеатров: {brand}
    Кинотеатр: {hall[0]}
    По адресу: {hall_data[0]}
    Метро: {hall_data[1]}
    Телефон: {hall_data[2]}
    Фильм: {film}
    Продолжительность: {film_data[0]} мин
    Жанры: {film_data[1]}
    Сеансы:'''
    for i in sessions_data:
        info += f"\n в формате {i[0]}: {i[1]}"
    return info

In [4]:
def keys(buttons=[], brand = None, hall = None, date = None, session = None, next_ = 0):   
    keyboard = {"one_time": True}
    buttons = buttons[30*next_:]
    if next_:
        payload={'brand':brand,'hall':hall,'date':date,'session':session,'next':next_ - 1}
        button_previous={"action": {"type": "text","payload": payload,"label": 'Назад'},
                        "color": "negative"}
    else:
        button_previous = None
    if len(buttons) > 30:
        payload={'brand':brand,'hall':hall,'date':date,'session':session,'next':next_ + 1}
        button_next={"action": {"type": "text","payload": payload,"label": 'Далее'},
                     "color": "positive"}
    else:
        button_next = None
    buttons = buttons[:30]
    li_buttons=[]
    for i, button in enumerate(buttons):
        payload={'brand':brand,'hall':hall,'date':date,'session':session,'next':0}
        if not payload['brand']:
            payload['brand'] = button
        elif not payload['hall']:
            payload['hall'] = button
        elif not payload['date']:
            payload['date'] = button
        else:
            payload['session']=button
        button = {"action": {"type": "text","payload": payload,"label": next_*30 + i + 1},
                "color": "secondary"}
        li_buttons.append(button)  
    buttons=[]
    while li_buttons:  
        buttons.append(li_buttons[:4])
        li_buttons = li_buttons[4:]
    if button_previous:
        buttons.append([button_previous])
    if button_next:
        buttons.append([button_next])
    button = {"action": {"type": "text","payload": None,"label": 'В меню'},
                "color": "primary"}
    buttons.append([button])
    keyboard["buttons"]= buttons 
    keyboard = str(json.dumps(keyboard))
    return keyboard

In [ ]:
token = "8b183ddf516b74eda6c20d66d0f703ea160e85d0740777e613c6b03dbd7b81e2b606b0f84762d8a289b0b"
vk = vk_api.VkApi(token=token)
longpoll = VkLongPoll(vk)
print("Server started")
brands = [el[0] for el in cursor.execute("select name from brand").fetchall()]
for event in longpoll.listen():
    if event.type == VkEventType.MESSAGE_NEW:
        if event.to_me:
            request = event.text.lower()
            payload=json.loads(event.extra_values.get('payload','""'))
            if not(payload):
                write_msg(event.user_id, 'Выберите  сеть кинотеатров:\n'+'\n'.join([str(i + 1) + '. ' + el for i, el in enumerate(brands)]),keys(brands))
            elif payload['brand'] and not payload['hall']:
                next_=payload['next']
                brand=payload['brand']
                hall=halls(brand)
                write_msg(event.user_id,'Выберите кинотеатр\n'+'\n'.join([str(i + 1) + '. ' + el[0] for i, el in enumerate(halls(brand))]),keys(hall, brand, next_=next_))
            elif payload['brand'] and payload['hall'] and not payload['date']:
                next_=payload['next']
                brand=payload['brand']
                hall=payload['hall']
                date=dates(hall)
                write_msg(event.user_id,'Выберите дату\n'+'\n'.join([str(i + 1) + '. ' + el for i,el in enumerate(date)]),keys(date,brand,hall,next_=next_))
            elif payload['brand'] and payload['hall'] and payload['date'] and not payload['session']:
                next_=payload['next']
                brand=payload['brand']
                hall=payload['hall']
                date=payload['date']
                session =conc_sessions(date,hall)
                write_msg(event.user_id,'Выберите фильм\n'+'\n'.join([str(i + 1) + '. ' + el for i,el in enumerate(session)]),keys(session,brand,hall,date,next_=next_))
            else:
                brand=payload['brand']
                hall=payload['hall']
                date=payload['date']
                session=payload['session']
                text=data(brand,hall,date,session)
                write_msg(event.user_id,text,keys())

Server started
